# Orthogonal Cutting : Simplest Model

In [2]:
import numpy as np

## Force model

Equations from *Metal Cutting Mechanics and Material Behavior*, Serge Jaspers, 1999 PhD Thesis

<img src="./ocutting-forces.jpg" width="400">

| Symbol     | Variable | Meaning                   | Relevant Concepts |Units|
|-----------------------------------------------------------------------------|
| b         | b         | Cutting width (not shown) |                   | [Meter](http://qudt.org/vocab/unit#Meter) |
| $h_{feed}$| h_feed    | Feed rate (diagram shows this as h)|          | [Meter](http://qudt.org/vocab/unit#Meter) |   
| $h$       | h         | Undeformed chip width (diagram shows this as $h_c$)||[Meter](http://qudt.org/vocab/unit#Meter) |    
| $\alpha$  | alpha     | Rake angle                |                   |[Radian](http://qudt.org/vocab/unit#Radian) |
| $\varphi$ | phi       | Shear angle               |                   |[Radian](http://qudt.org/vocab/unit#Radian) | 
| $F_s$     | F_s       | Shear force               |                   |[Newton](http://qudt.org/vocab/unit#Newton) |
| $F_V$     | F_V       | Cutting force             |                   |[Newton](http://qudt.org/vocab/unit#Newton) |     
| $F_T$     | F_T       | Thrust force              |                   |[Newton](http://qudt.org/vocab/unit#Newton) |
| $\beta$   | beta      | Friction angle (Eqn 2.8)  |                   |[Radian](http://qudt.org/vocab/unit#Radian) |
| $\tau_F$  | tau_F     | Shear flow stress         |                   |[Pascal](http://qudt.org/vocab/unit#Pascal) |

### Simplifications
* Orthogonal cutting with cutting width much larger than chip thickness ==> plain strain
* Perfectly sharp tool ==> no contact between clearance face and and workpiece.
* Continuous chip, no built-up edge
* Shear zone is plane and stress in this shear plane is uniform

### Notes
Force values are very sensitive to values of phi (and I'm guessing on phi): 
  * phi = pi/4 ==> F_T = 0, F_V = 517 newtons
  * phi = pi/3 ==> T_F is negative!
  * phi = pi/5 ==> F_V = 955 and F_T = 310 newtons. These seem to be reasonable. See for example see Table 1 in [1]. 

In [113]:
phi    = np.pi / 5   # Guessing. Need force values  -- with pi/4 F_T is zero!
alpha  = np.pi / 32  # 5 degree positive rake angle
h_feed = 0.000051    # Feed rate, m/sec using Ozel & Altan 
b = .005             # Cutting width (width of tool face) in meters
h = h_feed / np.tan(phi) + h_feed * np.sin(alpha) # undeformed chip width
print(h)

7.51943521008e-05


Eqn 2.8 (solving for beta):

$\varphi = \frac{\pi}{4} - \frac{\beta}{2} + \frac{\alpha}{2}$           


In [114]:
beta   = (np.pi / 2) + alpha - (2* phi) # Eqn 2.8
tau_F  = 923000000.0 # Pascals Ozel & Altan Table 3, 0.051mm/rev feed, 550m/min speed, P20 mold steel.

### Calculate principal cutting force (opposite direction of V)
Eqn 2.5:

$F_V = \tau_F b h \frac{cos(\beta - \alpha)}{cos(\varphi + \beta - \alpha)sin \varphi}$


In [115]:
def calc_F_V (alpha, beta, phi, tau_F, b, h): # Answer in newtons
    return (tau_F * b * h * (np.cos(beta-alpha) / (np.cos(phi+beta-alpha)*np.sin(phi))))
    

In [116]:
calc_F_V(alpha, beta, phi, tau_F, b, h) # 517 newtons (at phi=pi/4). Seems OK to me.

955.26943453144781

### Calculate thrust force (perpendicular to principal cutting force)
Eqn 2.6:

$F_T = \tau_F b h \frac{sin(\beta - \alpha)}{cos(\varphi + \beta - \alpha)sin \varphi}$

In [117]:
def calc_F_T (alpha, beta, phi, tau_F, b, h):
    return (tau_F * b * h * (np.sin(beta-alpha) / (np.cos(phi+beta-alpha)*np.sin(phi))))

In [118]:
calc_F_T(alpha, beta, phi, tau_F, b, h) # 310 newtons (at phi = pi/5 )

310.38585448853831

In [122]:
print("beta =", beta, "alpha = ", alpha)
np.sin(beta-alpha)

beta = 0.41233403578366046 alpha =  0.09817477042468103


0.30901699437494751

# References

[1] V. S. Sharma, S. Dhiman, R. Sehgal, and S. K. Sharma, “Estimation of cutting forces and surface roughness for hard turning using neural networks,” J. Intell. Manuf., vol. 19, no. 4, pp. 473–483, 2008.